In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import re
import random
import ml_metrics as metrics

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, coo_matrix

from catboost import CatBoostClassifier

pd.set_option('max_columns', 100)

In [3]:
train = pd.read_csv("../Data/train_go05W65.csv")
test = pd.read_csv("../Data/test_VkM91FT.csv")
sample_submission = pd.read_csv("../Data/sample_submission_kF044ur.csv")

In [4]:
train.Product_Holding_B1 = train.Product_Holding_B1.map(lambda x : re.findall("P\d+", x))
train.Product_Holding_B2 = train.Product_Holding_B2.map(lambda x : re.findall("P\d+", x))
test.Product_Holding_B1 = test.Product_Holding_B1.map(lambda x : re.findall("P\d+", x))

In [5]:
b1_products = sorted(list(set(sum(train.Product_Holding_B1.to_list(), []))))
b2_products = sorted(list(set(sum(train.Product_Holding_B2.to_list(), []))))

In [6]:
for p in b1_products:
    train[p] = train.Product_Holding_B1.map(lambda x : 1 if p in x else 0)
    test[p] = test.Product_Holding_B1.map(lambda x : 1 if p in x else 0)

In [7]:
train_labels = train[['Customer_ID', 'Product_Holding_B2']]
test_labels = test[['Customer_ID']]

for p in b2_products:
    train_labels[p] = train_labels.Product_Holding_B2.map(lambda x : 1 if p in x else 0)
    test_labels[p] = 0
train_labels.drop('Product_Holding_B2', axis = 1, inplace = True)
train_labels = train_labels.melt(id_vars=['Customer_ID']).rename(columns = {'variable' : 'product', 'value' : 'target'})
test_labels = test_labels.melt(id_vars=['Customer_ID']).rename(columns = {'variable' : 'product'}).drop('value', axis =1)
train1 = train.merge(train_labels, on = 'Customer_ID')
test1 = test.merge(test_labels, on = 'Customer_ID')


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [8]:
product_counts = train[b1_products].append(test[b1_products]).sum().to_dict()
train1['product_counts'] = train1['product'].map(product_counts)
test1['product_counts'] = test1['product'].map(product_counts)

In [9]:
count_matrix = train[b1_products].append(test[b1_products])
count_matrix = pd.DataFrame(np.matmul(count_matrix.values.T, count_matrix.values), columns = b1_products)
count_matrix['product'] = b1_products
count_matrix = count_matrix.set_index('product')

In [10]:
train1['pair_count_max'] = [max([count_matrix[p][row.product] for p in row.Product_Holding_B1]) 
                            for row in train1.itertuples()]
test1['pair_count_max'] = [max([count_matrix[p][row.product] for p in row.Product_Holding_B1]) 
                           for row in test1.itertuples()]

train1['pair_count_min'] = [min([count_matrix[p][row.product] for p in row.Product_Holding_B1]) 
                            for row in train1.itertuples()]
test1['pair_count_min'] = [min([count_matrix[p][row.product] for p in row.Product_Holding_B1]) 
                           for row in test1.itertuples()]

In [11]:
train1['product_count_max'] = train1['Product_Holding_B1'].map(lambda x : max([product_counts[p] for p in x]))
train1['product_count_min'] = train1['Product_Holding_B1'].map(lambda x : min([product_counts[p] for p in x]))

test1['product_count_max'] = test1['Product_Holding_B1'].map(lambda x : max([product_counts[p] for p in x]))
test1['product_count_min'] = test1['Product_Holding_B1'].map(lambda x : min([product_counts[p] for p in x]))


In [12]:
train1['Gender_Male'] = 1*(train1['Gender'] == "Male")
test1['Gender_Male'] = 1*(test1['Gender'] == "Male")

train1['City_Category_C1'] = 1*(train1.City_Category == "C1")
test1['City_Category_C1'] = 1*(test1.City_Category == "C1")

train1['Customer_Category_S1'] = 1*(train1.Customer_Category == "S1")
train1['Customer_Category_S2'] = 1*(train1.Customer_Category == "S2")
test1['Customer_Category_S1'] = 1*(test1.Customer_Category == "S1")
test1['Customer_Category_S2'] = 1*(test1.Customer_Category == "S2")

In [13]:
train1['Product_Holding_B1_len'] = train1['Product_Holding_B1'].map(len)
test1['Product_Holding_B1_len'] = test1['Product_Holding_B1'].map(len)

In [14]:
train1['self'] = [row.product in row.Product_Holding_B1 for row in train1.itertuples()]
test1['self'] = [row.product in row.Product_Holding_B1 for row in test1.itertuples()]

In [15]:
train1 = train1[~((train1['self'] == 1) & (train1['product'] != "P00"))]
test1 = test1[~((test1['self'] == 1) & (test1['product'] != "P00"))]

In [16]:
train1['vintage_by_age'] = train1['Vintage'] / train1['Age']
test1['vintage_by_age'] = test1['Vintage'] / test1['Age']

In [17]:
train1['id'] = train1['Customer_ID'].map(lambda x : int(x.replace("CC", "")))
test1['id'] = test1['Customer_ID'].map(lambda x : int(x.replace("CC", "")))

In [19]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, coo_matrix

df = train1[['id'] + b1_products].append(test1[['id'] + b1_products]).drop_duplicates()
df['id'] = df['id'] // 10
df = df.groupby('id').sum()
                                        
R = coo_matrix(df.values).asfptype()
U, sigma, Vt = svds(R, k = 8)
df1 = pd.DataFrame(np.matmul(U, Vt))
df1['id'] = df.index
df1 = df1.set_index('id')
df1.columns = b1_products
svd_score = df1.to_dict()

train1['svd_score1'] = [svd_score[row.product][row.id //10] for row in train1.itertuples()]
test1['svd_score1'] = [svd_score[row.product][row.id // 10] for row in test1.itertuples()]


In [20]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, coo_matrix

df = train1[['id'] + b1_products].append(test1[['id'] + b1_products]).drop_duplicates()
df['id'] = df['id'] // 100
df = df.groupby('id').sum()
                                        
R = coo_matrix(df.values).asfptype()
U, sigma, Vt = svds(R, k = 8)
df1 = pd.DataFrame(np.matmul(U, Vt))
df1['id'] = df.index
df1 = df1.set_index('id')
df1.columns = b1_products
svd_score = df1.to_dict()

train1['svd_score2'] = [svd_score[row.product][row.id //100] for row in train1.itertuples()]
test1['svd_score2'] = [svd_score[row.product][row.id // 100] for row in test1.itertuples()]


In [21]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix, coo_matrix

df = train1[['id'] + b1_products].append(test1[['id'] + b1_products]).drop_duplicates()
df['id'] = df['id'] // 1000
df = df.groupby('id').sum()
                                        
R = coo_matrix(df.values).asfptype()
U, sigma, Vt = svds(R, k = 8)
df1 = pd.DataFrame(np.matmul(U, Vt))
df1['id'] = df.index
df1 = df1.set_index('id')
df1.columns = b1_products
svd_score = df1.to_dict()

train1['svd_score3'] = [svd_score[row.product][row.id //1000] for row in train1.itertuples()]
test1['svd_score3'] = [svd_score[row.product][row.id // 1000] for row in test1.itertuples()]


In [22]:
target_counts = train1[train1['target'] == 1]['product'].value_counts().to_dict()

train1['product_target_count'] = train1['product'].map(target_counts)
test1['product_target_count'] = test1['product'].map(target_counts)


In [23]:
b1_count = train.append(test).Product_Holding_B1.astype(str).value_counts().to_dict()
train1['b1_count'] = train1['Product_Holding_B1'].astype(str).map(b1_count)
test1['b1_count'] = test1['Product_Holding_B1'].astype(str).map(b1_count)


In [24]:
train1['product1'] = train1['product'].map(lambda x : int(x.replace("P", "")))
test1['product1'] = test1['product'].map(lambda x : int(x.replace("P", "")))

In [25]:
train1.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2', 'P00',
       'P1', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18',
       'P19', 'P2', 'P20', 'P21', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9',
       'product', 'target', 'product_counts', 'pair_count_max',
       'pair_count_min', 'product_count_max', 'product_count_min',
       'Gender_Male', 'City_Category_C1', 'Customer_Category_S1',
       'Customer_Category_S2', 'Product_Holding_B1_len', 'self',
       'vintage_by_age', 'id', 'svd_score1', 'svd_score2', 'svd_score3',
       'product_target_count', 'b1_count', 'product1'],
      dtype='object')

In [34]:
indep_vars = [
    'Age',
    'Vintage', 'Is_Active', 'P00',
       'P1', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18',
       'P19', 'P2', 'P20', 'P21', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'product_counts', 'pair_count_max',
              'Gender_Male',
       'City_Category_C1', 'Customer_Category_S1', 'Customer_Category_S2', 'Product_Holding_B1_len',
             'vintage_by_age', 'product_count_max',
       'product_count_min', 'pair_count_min', 'svd_score1', 'svd_score2', 
    'svd_score3',
    'product_target_count', 'b1_count', 'id',
              'product1'
             ]
len(indep_vars)

43

In [35]:
clf = CatBoostClassifier(
    iterations=8000,
    eval_metric = 'AUC',
    learning_rate = 0.03,
    verbose=500,
)

random.seed(2)
dev_ids = random.sample(train.Customer_ID.to_list(), 30000)
dev = train1[train1.Customer_ID.isin(dev_ids)]
val = train1[train1.Customer_ID.isin(dev_ids) == False]


clf.fit(
    dev[indep_vars], dev.target,
    eval_set=(val[indep_vars], val.target),
)

0:	test: 0.8852196	best: 0.8852196 (0)	total: 109ms	remaining: 14m 29s
500:	test: 0.9813657	best: 0.9813657 (500)	total: 22.8s	remaining: 5m 41s
1000:	test: 0.9834726	best: 0.9834726 (1000)	total: 44.5s	remaining: 5m 11s
1500:	test: 0.9842965	best: 0.9842965 (1500)	total: 1m 9s	remaining: 5m
2000:	test: 0.9846385	best: 0.9846385 (1999)	total: 1m 33s	remaining: 4m 40s
2500:	test: 0.9847480	best: 0.9847486 (2481)	total: 2m 4s	remaining: 4m 34s
3000:	test: 0.9848392	best: 0.9848392 (3000)	total: 2m 32s	remaining: 4m 14s
3500:	test: 0.9849212	best: 0.9849212 (3497)	total: 2m 53s	remaining: 3m 43s
4000:	test: 0.9849610	best: 0.9849610 (4000)	total: 3m 14s	remaining: 3m 14s
4500:	test: 0.9849908	best: 0.9849925 (4432)	total: 3m 35s	remaining: 2m 47s
5000:	test: 0.9850037	best: 0.9850054 (4805)	total: 3m 57s	remaining: 2m 22s
5500:	test: 0.9850316	best: 0.9850327 (5455)	total: 4m 17s	remaining: 1m 57s
6000:	test: 0.9850479	best: 0.9850502 (5920)	total: 4m 38s	remaining: 1m 32s
6500:	test: 0.9

In [36]:
val['pred_cat'] = clf.predict_proba(val[indep_vars])[:,1]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df = val.sort_values(by = ['Customer_ID', 'pred_cat'], ascending = False)
actuals = df[df.target == 1][['Customer_ID', 'product']].groupby('Customer_ID').agg(list)['product'].to_list()
predicted = df.groupby('Customer_ID').head(3).groupby('Customer_ID').agg(list)['product'].to_list()
metrics.mapk(actuals, predicted, k=3)

0.8666214937188093

In [38]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric': {'auc'},
    'num_leaves': 96,
    'learning_rate': 0.01,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'verbose': 1,
    'min_data_in_leaf' : 1,
    'max_bin' : 255,
    'lambda_l1' : 0.00002,
    'lambda_l2' : 0.00001,
    'min_gain_to_split' : 0.001
}

# random.seed(2)
# dev_ids = random.sample(train.Customer_ID.to_list(), 30000)
# dev = train1[train1.Customer_ID.isin(dev_ids)]
# val = train1[train1.Customer_ID.isin(dev_ids) == False]
lgb_dev = lgb.Dataset(dev[indep_vars], dev['target'])
lgb_val = lgb.Dataset(val[indep_vars], val['target'])

model = lgb.train(params, lgb_dev, num_boost_round = 5000, valid_sets = (lgb_val), early_stopping_rounds = 500,
                 verbose_eval = 100)

Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.977776
[200]	valid_0's auc: 0.980603
[300]	valid_0's auc: 0.982151
[400]	valid_0's auc: 0.983089
[500]	valid_0's auc: 0.983619
[600]	valid_0's auc: 0.984011
[700]	valid_0's auc: 0.98426
[800]	valid_0's auc: 0.984455
[900]	valid_0's auc: 0.984576
[1000]	valid_0's auc: 0.984667
[1100]	valid_0's auc: 0.984737
[1200]	valid_0's auc: 0.984801
[1300]	valid_0's auc: 0.98485
[1400]	valid_0's auc: 0.984887
[1500]	valid_0's auc: 0.984921
[1600]	valid_0's auc: 0.98495
[1700]	valid_0's auc: 0.984982
[1800]	valid_0's auc: 0.984995
[1900]	valid_0's auc: 0.985008
[2000]	valid_0's auc: 0.985037
[2100]	valid_0's auc: 0.985046
[2200]	valid_0's auc: 0.985063
[2300]	valid_0's auc: 0.985069
[2400]	valid_0's auc: 0.985082
[2500]	valid_0's auc: 0.985096
[2600]	valid_0's auc: 0.985107
[2700]	valid_0's auc: 0.985113
[2800]	valid_0's auc: 0.985121
[2900]	valid_0's auc: 0.98513
[3000]	valid_0's auc: 0.985134
[3100]	valid_0's auc:

In [39]:
val['pred_lgb'] = model.predict(val[indep_vars])
df = val.sort_values(by = ['Customer_ID', 'pred_lgb'], ascending = False)
actuals = df[df.target == 1][['Customer_ID', 'product']].groupby('Customer_ID').agg(list)['product'].to_list()
predicted = df.groupby('Customer_ID').head(3).groupby('Customer_ID').agg(list)['product'].to_list()
metrics.mapk(actuals, predicted, k=3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.8675177823667756

In [51]:
val['pred'] = (val['pred_cat']**2 * 0.05 + val['pred_lgb']**2 * 0.95).map(np.sqrt)
df = val.sort_values(by = ['Customer_ID', 'pred'], ascending = False)
actuals = df[df.target == 1][['Customer_ID', 'product']].groupby('Customer_ID').agg(list)['product'].to_list()
predicted = df.groupby('Customer_ID').head(3).groupby('Customer_ID').agg(list)['product'].to_list()
metrics.mapk(actuals, predicted, k=3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.8675966557677967

In [41]:
clf = CatBoostClassifier(
    iterations=7777,
    learning_rate = 0.03,
    verbose=500,
    eval_metric = 'AUC'
)
clf.fit(train1[indep_vars], train1.target)

0:	total: 67.5ms	remaining: 8m 44s
500:	total: 26.7s	remaining: 6m 27s
1000:	total: 53.7s	remaining: 6m 3s
1500:	total: 1m 19s	remaining: 5m 33s
2000:	total: 1m 42s	remaining: 4m 55s
2500:	total: 2m 5s	remaining: 4m 25s
3000:	total: 2m 31s	remaining: 4m 1s
3500:	total: 2m 55s	remaining: 3m 34s
4000:	total: 3m 19s	remaining: 3m 8s
4500:	total: 3m 42s	remaining: 2m 41s
5000:	total: 4m 4s	remaining: 2m 15s
5500:	total: 4m 28s	remaining: 1m 51s
6000:	total: 4m 50s	remaining: 1m 26s
6500:	total: 5m 12s	remaining: 1m 1s
7000:	total: 5m 35s	remaining: 37.2s
7500:	total: 5m 57s	remaining: 13.2s
7776:	total: 6m 9s	remaining: 0us


In [42]:
lgb_train = lgb.Dataset(train1[indep_vars], train1['target'])
model = lgb.train(params, lgb_train, num_boost_round = 3690)

In [54]:
test1['pred_cat'] = clf.predict_proba(test1[indep_vars])[:,1]
test1['pred_lgb'] = model.predict(test1[indep_vars])
test1['pred'] = (test1['pred_cat']* 0.5 + test1['pred_lgb']* 0.5)

In [55]:
test1.sort_values(by = ['Customer_ID', 'pred'], ascending = False).groupby('Customer_ID').head(3).groupby(
    'Customer_ID').agg(list)['product'].reset_index().rename(columns = {"product" : "Product_Holding_B2"}).to_csv("./submission.csv", index = False)